In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
from matplotlib import figure
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
from sklearn.metrics import fbeta_score

In [112]:
train = pd.read_csv('AE_Deploy\AE_North1.csv')

In [113]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34364 entries, 0 to 34363
Data columns (total 23 columns):
 #   Column                                            Non-Null Count  Dtype  
---  ------                                            --------------  -----  
 0   MH_AandE_Diagnosis                                34364 non-null  int64  
 1   MH_OpenToMHSDSAtTimeOfAEAttendance                34364 non-null  int64  
 2   IMD                                               33530 non-null  float64
 3   Index_Of_Multiple_Deprivation_Decile_Description  33530 non-null  object 
 4   Sex                                               34364 non-null  int64  
 5   Ethnic_Category                                   34364 non-null  object 
 6   EC_Department_Type                                34364 non-null  int64  
 7   Arrival_Month                                     34364 non-null  int64  
 8   Age                                               34364 non-null  int64  
 9   Age_Band         

In [114]:
#Cleaning and replacing
train['Sex'].replace('1', '1', inplace=True)
train['Sex'].replace('2', '2', inplace=True)
train['Sex'].replace('9', '9', inplace=True)
train['Sex'].fillna('1', inplace=True)
train["IMD"].fillna("5", inplace = True)
train["Der_Number_EC_Treatment"].fillna("1", inplace = True)
train["EC_Investigation_01"].fillna("252167001", inplace = True)
train["EC_Treatment_01"].fillna("88140007", inplace = True)
train["Der_Number_EC_Treatment"].fillna("1", inplace = True)
train["AEA_Investigation_01"].fillna("5", inplace = True)
train["AEA_Treatment_01"].fillna("21", inplace = True)
train["Der_Number_AEA_Treatment"].fillna("1", inplace = True)

In [115]:
dfmodel = train[['MH_AandE_Diagnosis',
  'MH_OpenToMHSDSAtTimeOfAEAttendance',
  'IMD',
  'Age',
  'Sex',
  'Ethnic_Category',
  'Age_Band',
  'Der_Number_AEA_Diagnosis',
  'Der_Number_EC_Diagnosis',
  'Der_Number_AEA_Investigation',
  'Der_Number_EC_Investigation',                        
  'Der_Number_AEA_Treatment',                         
  'Der_Number_EC_Treatment']]
#'Der_Number_EC_Investigation',
 # 'Der_Number_AEA_Treatment',
  #'Der_Number_EC_Treatment',

In [116]:
dfmodel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34364 entries, 0 to 34363
Data columns (total 13 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   MH_AandE_Diagnosis                  34364 non-null  int64 
 1   MH_OpenToMHSDSAtTimeOfAEAttendance  34364 non-null  int64 
 2   IMD                                 34364 non-null  object
 3   Age                                 34364 non-null  int64 
 4   Sex                                 34364 non-null  int64 
 5   Ethnic_Category                     34364 non-null  object
 6   Age_Band                            34364 non-null  object
 7   Der_Number_AEA_Diagnosis            34364 non-null  int64 
 8   Der_Number_EC_Diagnosis             34364 non-null  int64 
 9   Der_Number_AEA_Investigation        34364 non-null  int64 
 10  Der_Number_EC_Investigation         34364 non-null  int64 
 11  Der_Number_AEA_Treatment            34364 non-null  in

In [117]:
#Dummy values for model preparation
a = pd.get_dummies(dfmodel['Ethnic_Category'], drop_first=True)
b = pd.get_dummies(dfmodel['IMD'], drop_first=True)
c = pd.get_dummies(dfmodel['Age_Band'], drop_first=True)
#d = pd.get_dummies(dfmodel['Sex'], drop_first=True)
#e = pd.get_dummies(dfmodel['EC_Investigation_01'], drop_first=True)
#f = pd.get_dummies(dfmodel['EC_Treatment_01'], drop_first=True)
#g = pd.get_dummies(dfmodel['AEA_Investigation_01'], drop_first=True)
#h = pd.get_dummies(dfmodel['HRG_Desc'], drop_first=True)
#i = pd.get_dummies(dfmodel['AEA_Treatment_01'], drop_first=True)
#j = pd.get_dummies(dfmodel['ICBName'], drop_first=True)
#k = pd.get_dummies(dfmodel['MH_OpenToMHSDSAtTimeOfAEAttendance'], drop_first=True)

In [118]:
dfmodel.drop(['IMD','Ethnic_Category','Age_Band'],
             axis=1,inplace=True)

C:\Users\aatif\AppData\Local\Temp\ipykernel_35164\1426948470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfmodel.drop(['IMD','Ethnic_Category','Age_Band'],


In [119]:
dfmodel = pd.concat([dfmodel,a,b,c],axis=1)

In [120]:
dfmodel.head()

,MH_AandE_Diagnosis,MH_OpenToMHSDSAtTimeOfAEAttendance,Age,Sex,Der_Number_AEA_Diagnosis,Der_Number_EC_Diagnosis,Der_Number_AEA_Investigation,Der_Number_EC_Investigation,Der_Number_AEA_Treatment,Der_Number_EC_Treatment,...,9.0,10.0,5,06-16yrs,17-40yrs,41-50yrs,51-60yrs,61-70yrs,71-84yrs,85+yrs
0,1,0,53,2,0,2,7,7,1,1,...,0,0,0,0,0,0,1,0,0,0
1,1,0,48,2,0,1,1,1,1,1,...,0,0,0,0,0,1,0,0,0,0
2,1,1,54,2,0,1,1,1,1,1,...,0,0,0,0,0,0,1,0,0,0
3,1,1,14,2,0,1,0,0,1,1,...,0,0,0,1,0,0,0,0,0,0
4,1,1,49,1,0,2,2,2,2,2,...,0,1,0,0,0,1,0,0,0,0


In [98]:
test = dfmodel[['MH_AandE_Diagnosis',
  'MH_OpenToMHSDSAtTimeOfAEAttendance',
  'IMD',
  'Age',
  'Ethnic_Category',
  'Der_Number_AEA_Diagnosis',
  'Der_Number_EC_Diagnosis',
  'Der_Number_AEA_Investigation',
  'Der_Number_EC_Investigation',                        
  'Der_Number_AEA_Treatment',                         
  'Der_Number_EC_Treatment']]

KeyError: "['IMD', 'Ethnic_Category'] not in index"

In [121]:
dftest = dfmodel[np.isfinite(dfmodel).all(1)]

In [124]:
X = dftest.drop('MH_OpenToMHSDSAtTimeOfAEAttendance',axis=1)
y = dftest['MH_OpenToMHSDSAtTimeOfAEAttendance']

In [126]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.20, random_state=101)

In [127]:
#Standardisation
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)

In [128]:
#RANDOM FOREST
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train,y_train)
predictions = rfc.predict(X_test)
print(classification_report(y_test,predictions))
print(confusion_matrix(y_test,predictions))
f = fbeta_score(y_test, predictions, beta=1.0)
print(f)

              precision    recall  f1-score   support

           0       0.68      0.67      0.67      4018
           1       0.54      0.55      0.54      2855

    accuracy                           0.62      6873
   macro avg       0.61      0.61      0.61      6873
weighted avg       0.62      0.62      0.62      6873

[[2699 1319]
 [1296 1559]]
0.5438688295831152


In [129]:
import pickle
pickle.dump(rfc, open('testmodel.pkl','wb'))

In [135]:
pickle.load(open('testmodel.pkl','rb'))

RandomForestClassifier()

In [125]:
#X = X[['MH_AandE_Diagnosis',
  'Sex',
  'Age',
  'Der_Number_AEA_Diagnosis',
  'Der_Number_EC_Diagnosis',
  'Der_Number_AEA_Investigation',
  'Der_Number_EC_Investigation',                        
  'Der_Number_AEA_Treatment',                         
  'Der_Number_EC_Treatment']]